In [64]:
from pymongo import MongoClient

client = MongoClient('192.168.9.169', 27017)

db = client.Fb_Raw

location = db.Fb_Location

In [59]:
def remove_accents(input_str):
    s1 = u'ÀÁÂÃÈÉÊÌÍÒÓÔÕÙÚÝàáâãèéêìíòóôõùúýĂăĐđĨĩŨũƠơƯưẠạẢảẤấẦầẨẩẪẫẬậẮắẰằẲẳẴẵẶặẸẹẺẻẼẽẾếỀềỂểỄễỆệỈỉỊịỌọỎỏỐốỒồỔổỖỗỘộỚớỜờỞởỠỡỢợỤụỦủỨứỪừỬửỮữỰựỲỳỴỵỶỷỸỹ'
    s0 = u'AAAAEEEIIOOOOUUYaaaaeeeiioooouuyAaDdIiUuOoUuAaAaAaAaAaAaAaAaAaAaAaAaEeEeEeEeEeEeEeEeIiIiOoOoOoOoOoOoOoOoOoOoOoOoUuUuUuUuUuUuUuYyYyYyYy'
    s = ''
    for c in input_str:
        if c in s1:
            s += s0[s1.index(c)]
        else:
            s += c
    return s

In [60]:
def levenshtein(s1, s2):
    if len(s1) < len(s2):
        return levenshtein(s2, s1)
    if len(s2) == 0:
        return len(s1)

    s1 = remove_accents(s1)
    s2 = remove_accents(s2)
    
    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1 
            deletions = current_row[j] + 1       
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row

    return previous_row[-1]

In [61]:
hihi = 0
class Node:
    nodeId = 0
    parent = None
    children = []
    items = set()
    
    def setParent(self, node):
        self.parent = node
        
    def setChildren(self, nodes):
        self.children = nodes
        
    def addItem(self, node):
        self.items.add(node)
        
    def setItem(self, nodes):
        self.items = nodes
        
    def __str__(self):
        st = "\n" + str(self.nodeId) + ": " + str(self.items) + "\n" + "Children of " + str(list(self.items)[0]) + ": " 
        if (len(self.children) > 0):
            for c in self.children:
                st += str(c) + "\n"
        return st
    
    def toDict(self):
        itsDict = dict()
        children = []
        for c in self.children:
            children.append(c.toDict())
        itsDict[list(self.items)[0]] = children
        return itsDict
    
    def __init__(self):
        global hihi
        self.nodeId = hihi
        self.items = set()
        self.parent = None
        self.children = []
        hihi +=1
    
    def numOfCharacter(self):
        sum = 0
        for i in self.items:
            sum += len(i)
        return sum
    
    def distance(self, item):
        sum = 0
        for i in self.items:
            sum += levenshtein(i, item)
        return sum/self.numOfCharacter()
    
    def satisfy(self, word, threshold):
        if word is None:
            return False
        if remove_accents(word) in remove_accents(str(self.items)):
            return True
        elif self.distance(word) < threshold:
            return True
        return False
    
    def satisfy2(self, word, threshold):
        for n in self.children:
            if n.satisfy(word, threshold):
                return n

In [93]:
def getNumberOfElement(listGroup):
    sum = 0
    for group in listGroup:
        sum += len(group.items)
    return sum

import random
def getOne(available):
    value = random.uniform(0.0, 1.0)
    n = getNumberOfElement(available)
    total = 0
    for a in available:
        total += len(a.items)/n
        if (total >= value):
            return a.nodeId
        
def findNodeById(nodes, nodeId):
    for i in range(len(nodes)):
        if nodes[i].nodeId == nodeId:
            return i
    print("out")

In [98]:
def fit(g, value, threshold):
    input_str = value.get("name").replace("[", "").replace("]", "")
    s = input_str.split(", ")
    s = s[::-1]
    
    available = []
    for n in g:
        if n.satisfy(s[0], threshold):
            available.append(n)
    if len(available) == 1:
        available[0].addItem(s[0])
        if len(s) > 1:
            s.remove(s[0])
            s = s[::-1]
            value["name"] = str(s).replace("\'", "")
            available[0].children = fit(available[0].children, value, threshold)    
    elif len(available) == 0:
        node = Node()
        node.addItem(s[0])
        if len(s) > 1:
            s.remove(s[0])
            s = s[::-1]
            value["name"] = str(s).replace("\'", "")
            node.children = fit(node.children, value, threshold)    
        g.append(node)
        # todo check children of node in g

    elif len(available) > 1:
        a = findNodeById(g, getOne(available))
        g[a].addItem(s[0])
#     todo check children of available
                
    return g

In [101]:
g = []
hihi = 0
threshold = 0.5
data = location.find().limit(10000)
import time
start = time.time()
for d in data:
    g = fit(g, d, threshold)
end = time.time()
print(end-start)

from pprint import pprint
for group in g:
    print("\ngroup: " + str(group.toDict()))

906.1430907249451

group: {'Vietnam': [{'Hanoi': []}, {'Hồ Chí Minh': [{'Cần Giờ': []}, {'Bình Chánh': []}, {'Hóc Môn': []}, {'Thành phố Hồ Chí Minh': []}, {'Go Vap': []}, {'Củ Chi': []}]}, {'Tuyên Quang': [{'Tuyên Quang': []}, {'Vàm Rầy (2)': []}, {'Long Hoa': []}, {'Mitho': []}, {'Châu Phong': []}, {'Chaudok': []}, {'Bình Thành (3)': []}, {'Ap Tay': []}, {'Ap Rạch Soi': []}, {'Soc Son': []}, {'Phu My': []}, {'Ap Tan Lap (1)': []}, {'Giong Ba Ninh': []}, {'Angiang': []}, {'Phu Quoc': []}, {'Ap My Hoa': []}, {'Duong Dong': []}, {'Hoa Dong': []}, {'Vinh Thanh': []}, {'Ap Hoa Loi': []}, {'Ba Hòn': []}, {'Ba Thê': []}, {'My Hoi': []}, {'Tay Phu': []}, {'Ap Khanh Binh': []}, {'Ap Son Tay': []}, {'Tan My': []}, {'Huyen Chau Thanh': []}, {'Xom Lo Gach': []}, {'Vam Lang': []}, {'Tan Hiep': []}, {'Ben Tranh': []}, {'Ba Chuc': []}, {'Kiên Long': []}, {'Hoa Binh': []}, {'Ap Phu Hoa': []}, {'Vinh Phong': []}, {'Trung An': []}, {'�Ông Thái': []}, {'Huu An': []}, {'Vinh Hoi Dong': []}, {'Lai Son': 


group: {'West Yorkshire': [{'Birstall': []}]}

group: {'Jinhua': []}

group: {'Ramnicu Sarat': []}

group: {'Pusztavám': []}

group: {'Kunsziget': []}

group: {'Ullo': []}

group: {'Mali Iđoš': []}

group: {'Puerto General San Martín': []}

group: {'Marcali': []}

group: {'Kozármisleny': []}

group: {'Bad Hönningen': []}

group: {'Königsfeld im Schwarzwald': []}

group: {'Zomergem': []}

group: {'Abony': []}

group: {'Homokszentgyörgy': []}

group: {'Mohács': []}

group: {'Kispest': []}

group: {'Prachatice': []}

group: {'Tuzsér': []}

group: {'Balatonfüred': []}

group: {'Kalocsa': []}

group: {'Sassandra': []}

group: {'Yeosu': []}

group: {'Mukundapur': []}

group: {'Imperia': []}

group: {'Fertoszentmiklos': []}

group: {'Gyomaendrod': []}

group: {'Lábatlan': []}

group: {'Nagyecsed': []}

group: {'Ötvöskónyi': []}

group: {'Poipet': []}

group: {'Terborg': []}

group: {'Anhembi': []}

group: {'Tlaltizapán': []}

group: {'Mezokovesd': []}

group: {'Värnamo': []}

group: {'Albufe